In [1]:
import tensorflow as tf
import numpy as np
import logging
from hooks import HistoryHook, EarlyStoppingHook, keras_model_to_estimator

/home/jenno/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.9.0'

In [3]:
history_hook = HistoryHook()

In [4]:
def estimator_fn(run_config, params):
    x_in = tf.keras.layers.Input(shape= (28, 28, 1))
    x = tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation='relu')(x_in)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (5, 5), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x_out = tf.keras.layers.Dense(10, activation= 'softmax')(x)

    model = tf.keras.models.Model(x_in, x_out)
    optimizer = tf.train.AdamOptimizer(learning_rate= params["learning_rate"])
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer)
    
    early_stopping_hook = EarlyStoppingHook(history_hook=history_hook, patience=1, metric_name='loss')
    

    estimator = keras_model_to_estimator(model = model,
                                         run_config = run_config,
                                         training_hooks= [early_stopping_hook],
                                         evaluation_hooks= [history_hook])
    
    return estimator

In [5]:
def _input_fn(mode, features, labels):
    dataset = tf.data.Dataset.from_tensor_slices( (features, labels))
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(60000))
    dataset = dataset.batch(128)
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    return dataset

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data('/home/jenno/Desktop/data/mnist/mnist.npz')

In [7]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [8]:
x_train = np.float32(x_train) / 255
x_test = np.float32(x_test) / 255


In [9]:
para_dic = {"learning_rate": 1e-4}
model_estimator = estimator_fn(tf.estimator.RunConfig(save_checkpoints_steps=600), para_dic)
logging.getLogger().setLevel(logging.INFO)
Exp = tf.contrib.learn.Experiment(model_estimator,
                                 train_input_fn= lambda: _input_fn(tf.estimator.ModeKeys.TRAIN, x_train, y_train),
                                 eval_input_fn= lambda: _input_fn(tf.estimator.ModeKeys.EVAL, x_test, y_test),
                                 min_eval_frequency= 600,
                                 train_steps= 20000)
Exp.train_and_evaluate()

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpwn0mca8j', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 600, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f79d6122cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Please switch to tf.estimator.train_and_evaluate. You will also have to convert to a tf.estimator.Estimator.
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:G

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-04:51:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpwn0mca8j/model.ckpt-4200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:HistoryHook: steps since the best: 0
INFO:tensorflow:Finished evaluation at 2018-08-14-04:51:05
INFO:tensorflow:Saving dict for global step 4200: global_step = 4200, loss = 0.044710908
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4200: /tmp/tmpwn0mca8j/model.ckpt-4200
INFO:tensorflow:Validation (step 4204): loss = 0.044710908, global_step = 4200
INFO:tensorflow:global_step/sec: 93.1

({'global_step': 6610, 'loss': 0.04037425}, [])